In [13]:
!pip install langchain
!pip install google-generativeai
!pip install gradio
!pip install langchain-google-genai
!pip install youtube-transcript-api

In [16]:
import gradio as gr
from youtube_transcript_api import YouTubeTranscriptApi
import google.generativeai as genai
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate, LLMChain

# Set the API key in Google Colab
from google.colab import userdata
api_key = userdata.get('Gemini_API_Key')

if not api_key:
    raise ValueError("API key for Gemini is not set. Please set the GEMINI_API_KEY environment variable.")

# Configure Gemini API
genai.configure(api_key=api_key)

def get_video_id(url):
    video_id = ""
    if "shorts/" in url:
        video_id = url.split("shorts/")[1].split("?")[0]
    elif "youtu.be/" in url:
        video_id = url.split("youtu.be/")[1].split("?")[0]
    elif "watch?v=" in url:
        video_id = url.split("watch?v=")[1].split("&")[0]
    return video_id

def extract_transcript(video_url):
    if video_url:
        video_id = get_video_id(video_url)
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            text = " ".join([t['text'] for t in transcript_list])
            return text
        except Exception as e:
            return f"Error retrieving transcript: {str(e)}"
    return "No video URL provided."

def generate_response(video_url, user_input, chat_history):
    if not user_input:
        return "Enter a URL...", chat_history

    try:
        # Extract transcript if a YouTube URL is provided
        transcript = extract_transcript(video_url) if video_url else ""

        # Create the LLM chain
        llm = ChatGoogleGenerativeAI(
            temperature=0.6,
            google_api_key=api_key,
            model='gemini-1.5-flash',
            max_output_tokens=2048,
            verbose=True,
        )

        memory = ConversationBufferMemory(memory_key="chat_history")

        instruction = f"""You are a helpful coding assistant. You solve user's coding questions.
                      Below is an instruction that describes a task. Write a response that appropriately completes the request.
                      User input: {user_input}
                      Transcript: {transcript}
                      Chat history of the user: {chat_history}
                      """

        prompt_template = PromptTemplate(
            input_variables=["chat_history", "user_input"],
            template = instruction
        )

        llm_chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True, memory=memory)
        output = llm_chain.run({"chat_history": chat_history, "user_input": user_input})

        chat_history.append({"user_input": user_input, "response": output})
        print(chat_history)
        return output, chat_history

    except Exception as e:
        error_message = f"Error: {str(e)}"
        print(error_message)  # Print detailed error message
        return error_message, chat_history

# Gradio Interface Setup
def launch_interface():
    with gr.Blocks() as demo:
        chat_history = gr.State([])
        video_url = gr.Textbox(label="YouTube URL (Optional)", placeholder="Enter YouTube URL here", interactive=True)
        query = gr.Textbox(label="Your Query", placeholder="Enter your query here", interactive=True)
        response_box = gr.Textbox(label="Response", placeholder="The bot's response will appear here", interactive=False)

        with gr.Row():
            submit_btn = gr.Button("Submit")
            clear_btn = gr.Button("Clear")

        with gr.Row():
            history_btn = gr.Button("Chat History")

        chat_history_display = gr.Markdown()
        back_btn = gr.Button("Back", visible=False)

        # Submit button logic
        submit_btn.click(fn=generate_response, inputs=[video_url, query, chat_history], outputs=[response_box, chat_history])

        # Clear button logic
        def clear_fields():
            return "", "", ""

        clear_btn.click(fn=clear_fields, inputs=[], outputs=[video_url, query, response_box])

        # Chat History and Back button logic
        def show_chat_history(history):
            return "\n\n".join([f"**User Query:** {c['user_input']} \n**Response:** {c['response']}" for c in history])

        history_btn.click(fn=show_chat_history, inputs=[chat_history], outputs=[chat_history_display])
        back_btn.click(fn=lambda: "", inputs=[], outputs=[chat_history_display])

        demo.launch()

# Launch the interface
launch_interface()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://629dada7bb8a22f63d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
